# Decode 'langauge' feature

In [10]:
import sys
sys.path.append('..')

from utils.cuda_cluster import client
from utils.util import plot_graphs
import core.config as conf
from utils.dataset import read_data, factorize_small_cardinality

import numpy as np
from tqdm import tqdm
from datetime import datetime 
import matplotlib.pyplot as plt

import tensorflow as tf
from transformers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [8]:
client

Client Scheduler: tcp://127.0.0.1:39629 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 33.47 GB


In [5]:
#random seed 
tf.random.set_seed(1234)
np.random.seed(1234)

In [89]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)

Downloading: 100%|██████████| 996k/996k [00:01<00:00, 818kB/s] 
Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 41.7kB/s]
Downloading: 100%|██████████| 1.96M/1.96M [00:01<00:00, 1.44MB/s]


## 1. Load data

In [11]:
# data_path = conf.raw_data_path + '*' # for all dataset
data_path = conf.raw_data_path + 'part-00175'
ori_df = read_data(data_path)

number of rows: 3033347


In [18]:
df = ori_df[['text_ tokens', 'language']]

In [19]:
df.head()

,text_ tokens,language
id,,
1,101\t12845\t10341\t55818\t13410\t10129\t23633\...,B8B04128918BBF54E2E178BFF1ABA833
2,101\t56898\t137\t181\t10123\t10410\t168\t168\t...,9FCF19233EAD65EA6E32C2E6DC03A444
3,101\t11916\t16818\t117\t10911\t35691\t10638\t1...,B0FA488F2911701DD8EC5B1EA5E322D8
4,101\t66676\t53622\t11499\t38356\t10703\t12843\...,1F73BB863A39DB62B4A55B7E558DB1E8
5,101\t56898\t137\t158\t168\t10111\t168\t100061\...,E7F038DE3EAD397AEC9193686C911677


## Get unique language

In [84]:
languages = df.compute().groupby('language')[['text_ tokens']].nth(-1)
languages = languages.reset_index()

In [85]:
tokens = list(map(lambda x: x.split('\t'), languages['text_ tokens'].to_array()))
decoded_tokens = []
n_languages = len(languages)

for i in range(n_languages):
    decoded_tokens.append(tokenizer.decode(tokens[i]))

In [86]:
languages['decoded_tweet_tokens'] = decoded_tokens

In [88]:
languages

,language,text_ tokens,decoded_tweet_tokens
0,00304D7356D6C64481190D708D8F739C,101\t56898\t137\t26747\t16724\t13034\t51427\t1...,[CLS] RT @ FarazWahab1 : اس پرچم کے سائے تلے ہ...
1,0BB2C843174730BA7D958C98B763A797,101\t638\t52988\t34748\t67457\t18555\t14120\t1...,[CLS] Ծաղկաձոր https : / / t. co / TSlacMT5JV ...
2,105008E45831ADE8AF1DB888319F422A,101\t56898\t137\t139\t50198\t11517\t10157\t741...,[CLS] RT @ BhoneMy91441280 : Followersနည်းနေတဲ...
3,10C6C994C2AD434F9D49D4BE9CFBC613,101\t56898\t137\t178\t10174\t49025\t11211\t113...,[CLS] RT @ jk100687 : Austro - Hungarian storm...
4,159541FA269CA8A9CDB93658CAEC4CA2,101\t56898\t137\t44286\t16105\t51658\t11373\t6...,[CLS] RT @ Nansy64953936 : Πάντα θα σε στηριζο...
...,...,...,...
61,E7F038DE3EAD397AEC9193686C911677,101\t100\t216\t3796\t2023\t15464\t17575\t99827...,[CLS] [UNK] ¶ 忍 トラみたいにこんな 技 使 って 欲 しかったなぁ ｡ [U...
62,E84BE2C963852FB065EE827F41A0A304,101\t45113\t118\t25587\t10137\t15788\t20794\t1...,[CLS] Southampton - Arsenal är första FA - cup...
63,F33767F7D7080003F403FDAB34FEB755,101\t100\t100\t100\t100\t100\t100\t100\t100\t1...,[CLS] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK...
64,F70598172AC4514B1E6818EA361AD580,101\t56898\t137\t21573\t11669\t10133\t168\t192...,[CLS] RT @ jensoo _ xoxo : rawr! [UNK] https :...
